In [2]:
import glob
import pickle
import numpy
from music21 import converter, instrument, note, chord, interval, pitch
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint


C:\Users\alecr\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
def get_notes(transpose=False):
    """ Get all the notes and chords from the midi files in the ./midi_songs directory """
    print('starting get_notes')
    notes = []

    for file in glob.glob("C:/Users/alecr/Projects/temp-data/skuldur-classical-piano-composer/*.mid"):
        s = converter.parse(file)
        
        if (transpose):
            k = s.analyze('key')
            #print(k)
            i = interval.Interval(k.tonic, pitch.Pitch('C'))
            midi = s.transpose(i)
            #print('piece was in ',k, 'now in ', midi.analyze('key'))
        else:
            midi = s
        #print(i)
        #print('s now in ', midi.analyze('key'))
        #pieces.append(sNew)

        #print("Parsing %s" % file)

        notes_to_parse = None

        try: # file has instrument parts
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[0].recurse() 
        except: # file has notes in a flat structure
            notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))

    #with open('data/notes', 'wb') as filepath:
        #pickle.dump(notes, filepath)
    print('done')
    return notes

In [4]:
notes = get_notes()

starting get_notes
done


pickle dump

In [5]:
fileObject = open("C:/Users/alecr/Projects/deepmusic/pickledump/notes",'wb')
pickle.dump(notes,fileObject)
fileObject.close()
print('dumped')

dumped


Interesting that there isn't much of a difference in the size of the set of notes between transposed and not. A better metric would be to see a frequency distribution of the notes and compare those. 

In [5]:
def prepare_sequences(notes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 100

    # get all pitch names
    pitchnames = sorted(set(item for item in notes))

     # create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    network_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    network_input = network_input / float(n_vocab)

    network_output = np_utils.to_categorical(network_output)

    return (network_input, network_output)

# Non transposing original

In [6]:

""" Train a Neural Network to generate music """
notes = get_notes(transpose=False)

# get amount of pitch names
n_vocab = len(set(notes))

network_input, network_output = prepare_sequences(notes, n_vocab)


print('notes and sequences prepared')

starting get_notes
done
notes and sequences prepared


In [7]:
network_input.shape

(57077, 100, 1)

In [1]:
#network_input[0]

In [9]:
len(notes)

57177

In [8]:
model = Sequential()
model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        return_sequences=True
))
model.add(Dropout(0.3))
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(512))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')


In [10]:
model.load_weights("C:/Users/alecr/Projects/deepmusic/models/weights-improvement-60-1.5352-bigger.hdf5")

In [14]:
filepath = "C:/Users/alecr/Projects/deepmusic/models/weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
checkpoint = ModelCheckpoint(
        filepath,
        monitor='loss',
        verbose=0,
        save_best_only=True,
        mode='min'
    )
callbacks_list = [checkpoint]


model.fit(network_input, network_output, epochs=50, batch_size=128, verbose=1, callbacks=callbacks_list)

Epoch 1/50
57077/57077 [==============================] - 209s 4ms/step - loss: 0.1970
Epoch 2/50
57077/57077 [==============================] - 209s 4ms/step - loss: 0.1953
Epoch 3/50
57077/57077 [==============================] - 209s 4ms/step - loss: 0.1981
Epoch 4/50
57077/57077 [==============================] - 209s 4ms/step - loss: 0.1971
Epoch 5/50
57077/57077 [==============================] - 206s 4ms/step - loss: 0.1896
Epoch 6/50
57077/57077 [==============================] - 207s 4ms/step - loss: 0.1919
Epoch 7/50
57077/57077 [==============================] - 208s 4ms/step - loss: 0.1945
Epoch 8/50
57077/57077 [==============================] - 901s 16ms/step - loss: 0.1910
Epoch 9/50
57077/57077 [==============================] - 2340s 41ms/step - loss: 0.1948
Epoch 10/50
57077/57077 [==============================] - 3743s 66ms/step - loss: 0.1901
Epoch 11/50
57077/57077 [==============================] - 209s 4ms/step - loss: 0.1987
Epoch 12/50
57077/57077 [===========

250 epochs so far

In [13]:
model.save_weights("C:/Users/alecr/Projects/deepmusic/models/lstm-original-weights-2.hdf5")
print('saved weights')

saved weights


# transposer half

In [4]:
filepath_t = "C:/Users/alecr/Projects/deepmusic/models/weights-improvement-transposing{epoch:02d}-{loss:.4f}-bigger.hdf5"
checkpoint = ModelCheckpoint(
        filepath_t,
        monitor='loss',
        verbose=0,
        save_best_only=True,
        mode='min'
    )
callbacks_list_t = [checkpoint]

In [4]:
transposed_notes = get_notes(transpose=True)

# get amount of pitch names
n_vocab_t = len(set(transposed_notes))

network_input_t, network_output_t = prepare_sequences(transposed_notes, n_vocab_t)


print('notes and sequences prepared')

starting get_notes
done
notes and sequences prepared


In [13]:
#pickle dump

noteFileObject = open("C:/Users/alecr/Projects/deepmusic/pickledump/transposed_notes",'wb')

In [14]:
pickle.dump(transposed_notes,noteFileObject)

In [7]:
pickle.dump(network_input_t,fileObject)

In [8]:
pickle.dump(network_output_t,fileObject)

In [9]:
fileObject.close()

In [6]:
model_t = Sequential()
model_t.add(LSTM(
        512,
        input_shape=(network_input_t.shape[1], network_input_t.shape[2]),
        return_sequences=True
))
model_t.add(Dropout(0.3))
model_t.add(LSTM(512, return_sequences=True))
model_t.add(Dropout(0.3))
model_t.add(LSTM(512))
model_t.add(Dense(256))
model_t.add(Dropout(0.3))
model_t.add(Dense(n_vocab_t))
model_t.add(Activation('softmax'))
model_t.compile(loss='categorical_crossentropy', optimizer='rmsprop')


In [7]:
model_t.load_weights("C:/Users/alecr/Projects/deepmusic/models//lstm-transposition-weights.hdf5")

currently on training round 2
60 + 60 epochs
   

In [11]:
model_t.fit(network_input_t, network_output_t, epochs=1, batch_size=64 , verbose=1)

Epoch 1/1
57077/57077 [==============================] - 354s 6ms/step - loss: 4.7006


In [12]:
model_t.save_weights("C:/Users/alecr/Projects/deepmusic/models//lstm-transposition-weights.hdf5")
print("saved")

saved


In [32]:
network_input.shape

(57077, 100, 1)

In [33]:
network_input_t.shape

(57077, 100, 1)

In [36]:
network_output.shape

(57077, 358)

In [37]:
network_output_t.shape

(57077, 344)

In [4]:
network_output[0]

NameError: name 'network_output' is not defined

In [15]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

['/job:localhost/replica:0/task:0/device:GPU:0']

In [16]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 1051151653148060444
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 6701754613
locality {
  bus_id: 1
  links {
  }
}
incarnation: 7915996438733296818
physical_device_desc: "device: 0, name: GeForce GTX 1070, pci bus id: 0000:01:00.0, compute capability: 6.1"
]
